# List of Sensors not Reporting
These sensors did not report in the 4 hours prior to when this report was run.

In [ ]:
# Import any necessary Python packages
from datetime import datetime, timedelta
import pandas as pd
from IPython.display import Markdown
import bmondata
import scrapbook as sb

In [ ]:
# Parameters that will be changed by the bmonreporter report-creating script.
# This cell must have a "parameters" tag.
server_web_address = 'https://bmon.analysisnorth.com'
org_id = 1

In [ ]:
# The report generator script needs to the know the Title of this report and where
# to sort it with respect to other reports.  Glue that information to this notebook.
sb.glue('title', 'Non-reporting Sensors')
sb.glue('sort_order', 90)

####################################################################################

# This code creates the list of sensors that are not reporting. Each non-reporting
# sensor is listed as a bullet item underneath the associated Building Name.
# Markdown is used to create the formatted text.
server = bmondata.Server(server_web_address)
result = ''

# Even though there is only one Organization ID set in the "parameters" cell above
# the server.organizations() method returns a list of organizations; thus you use a
# for loop to process the item(s) in the returned list. Ususally, this
# will be a list of one organization (e.g. if the ID is set to 1 by the report
# creator, the returned list will just have one dictionary describing the organization
# with the ID = 1).  However, the report creator also does one report where it passes
# in the special Organization ID of 0, causing server.organizations() to return
# a list of "All Organizations", one dictionary for each organization.  This report
# will do an analysis of the buildings from all organizations on the BMON server.
for org_info in server.organizations(org_id):
    for bldg_id, bldg_name in org_info['buildings']:
        result += f'\n\n### {bldg_name}'
        bldg_info = server.buildings(bldg_id)[0]
        for sensor_info in bldg_info['sensors']:
            s_id = sensor_info['sensor_id']
            tstart = str(datetime.utcnow() - timedelta(hours=4))
            try:
                reading_count = len(server.sensor_readings(s_id, start_ts=tstart, timezone='UTC'))
            except:
                reading_count = 0
            if reading_count==0:
                # Look up sensor title and add a bullet describing sensor
                try:
                    detailed_info = server.sensors(s_id)[0]
                    title = detailed_info['title']
                except:
                    title = "Unknown title"
                result += f"\n* {title} ({s_id})"
Markdown(result)